In [2]:
import numpy as np
import os
dtype = np.float32

import torch
import torch.nn as nn
import torchvision
from tqdm import tqdm
import os

from transformers import CLIPVisionModelWithProjection, AutoModelForCausalLM, LlamaForCausalLM
from transformers import AutoModel, AutoTokenizer, OPTForCausalLM, BloomForCausalLM
import numpy

from huggingface_hub import try_to_load_from_cache, _CACHED_NO_EXIST
from huggingface_hub import scan_cache_dir

import glob
import random
import json
import os

from datasets import load_dataset

import functools
import gc
from collections import defaultdict
from typing import List

import torch
import torch.nn as nn
# from tinychat.models import LlavaLlamaForCausalLM
from transformers.models.bloom.modeling_bloom import BloomForCausalLM
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from transformers.models.opt.modeling_opt import OPTForCausalLM

import numpy as np
from scipy.linalg import eigh

import torch

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")

def topk_eigenvectors(A, device):
    A = A.to(device)
    eigenvalues, eigenvectors = torch.linalg.eigh(A)
    eigenvalues = eigenvalues
    eigenvectors = eigenvectors

    # 고유값 행렬 (대각행렬)
    Lambda = torch.diag(eigenvalues)

    # 원래 행렬 복원 검증: A @ V ≈ V @ Lambda
    A_reconstructed = eigenvectors @ Lambda @ eigenvectors.T
    error = torch.norm(A - A_reconstructed)
    
    # assert error < 1e-4, f"Error: {error.item()}"
    print(f"재구성 오차 (||A - VΛV^T||): {error.item()}")
    return eigenvalues.cpu(), eigenvectors.cpu()

def get_named_linears(module):
    return {name: m for name, m in module.named_modules() if isinstance(m, nn.Linear)}

def get_blocks(model):
    if model.__class__.__name__ in ("LlamaForCausalLM", "Qwen2ForCausalLM"):
        layers = model.model.layers
    elif model.__class__.__name__ == "LlavaLlamaForCausalLM":
        # layers = [model.model.layers, model.model.vision_tower.vision_tower.vision_model.encoder.layers]
        layers = model.model.layers
    elif isinstance(model, OPTForCausalLM):
        layers = model.model.decoder.layers
    elif isinstance(model, BloomForCausalLM):
        layers = model.transformer.h
    elif "mpt" in str(model.__class__).lower():
        layers = model.transformer.blocks
    elif "falcon" in str(model.__class__).lower():
        layers = model.transformer.h
    elif "bigcode" in str(model.__class__).lower():
        layers = model.transformer.h
    elif "neox" in str(model.__class__).lower():
        layers = model.gpt_neox.layers
    elif model.__class__.__name__ == "LlavaLlamaModel":
        layers = model.llm.model.layers
    else:
        raise NotImplementedError(type(model))
    return layers

/home/jgryu/miniconda3/envs/nic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def flat_to_sym(V, N):
    A = torch.zeros(N, N, dtype=V.dtype, device=V.device)
    idxs = torch.tril_indices(N, N, device=V.device)
    A[idxs.unbind()] = V
    A[idxs[1, :], idxs[0, :]] = V
    return A

def regularize_H(H, n, sigma_reg):
    H.div_(torch.diag(H).mean())
    idx = torch.arange(n)
    H[idx, idx] += sigma_reg
    return H

In [4]:
quip_hess_base_path = '/home/jgryu/Weight_compression/Wparam_dataset/quip_hess'

model_list = os.listdir(quip_hess_base_path)
quip_hess_path = [os.path.join(quip_hess_base_path, model_name) for model_name in model_list]

sigma_reg = 1e-4

In [5]:
quip_hess_path

['/home/jgryu/Weight_compression/Wparam_dataset/quip_hess/llama3_8b_6144',
 '/home/jgryu/Weight_compression/Wparam_dataset/quip_hess/Hessians-Llama-2-13b-6144',
 '/home/jgryu/Weight_compression/Wparam_dataset/quip_hess/Hessians-Llama-2-7b-6144',
 '/home/jgryu/Weight_compression/Wparam_dataset/quip_hess/llama3.1_8b_6144']

In [6]:
# for quip_hess in quip_hess_path:
#     save_path = quip_hess.replace('quip_hess', f'quip_hess_eig_reg{sigma_reg}')
#     os.makedirs(save_path, exist_ok=True)
#     print(f'##### {quip_hess.split("/")[-1]} #####')
#     for i in tqdm(range(40)):
#         try:
#             hess_dict = {}
#             hess_dict['qkv'] = torch.load(f'{quip_hess}/{i}_qkv.pt', weights_only=False)
#             hess_dict['o'] = torch.load(f'{quip_hess}/{i}_o.pt', weights_only=False)
#             hess_dict['up'] = torch.load(f'{quip_hess}/{i}_up.pt', weights_only=False)
#             hess_dict['down'] = torch.load(f'{quip_hess}/{i}_down.pt', weights_only=False)
#         except:
#             continue

#         for k, h in hess_dict.items():
#             print(f'## layer{i}, {k} ##')
            
#             H = flat_to_sym(h['flatH'], h['n']).to(device)
#             mu = h['mu'].to(device)
#             H.add_(mu[None, :] * mu[:, None])
#             n_h = h['n']
#             H = regularize_H(H, n_h, sigma_reg)

#             eig = {}
#             s, q = topk_eigenvectors(H, device)
#             eig['eigenvalues'], eig['eigenvectors'] = s, q
#             torch.save(eig, f'{save_path}/{i}_{k}_eig.pt')
#             print(f"{(s[0]/s.sum()).item():.3f}, {(s[-1]/s.sum()).item():.3f}")
#             print(f"{q[-1].max().item():.3f}, {q[-1].min().item():.3f}")



In [10]:
for quip_hess in quip_hess_path:
    if '2-7b' not in quip_hess: continue
    save_path = quip_hess.replace('quip_hess', f'quip_hess_eig_reg{sigma_reg}')
    # os.makedirs(save_path, exist_ok=True)
    print(f'##### {quip_hess.split("/")[-1]} #####')
    for i in tqdm(range(40)):
        try:
            hess_dict = {}
            hess_dict['qkv'] = torch.load(f'{save_path}/{i}_qkv_eig.pt', weights_only=False)
            hess_dict['o'] = torch.load(f'{save_path}/{i}_o_eig.pt', weights_only=False)
            hess_dict['up'] = torch.load(f'{save_path}/{i}_up_eig.pt', weights_only=False)
            hess_dict['down'] = torch.load(f'{save_path}/{i}_down_eig.pt', weights_only=False)
        except:
            continue

        for k, h in hess_dict.items():
            print(f'## layer{i}, {k} ##')
            s, q = h['eigenvalues'], h['eigenvectors']
            # print(f"{(s[0]/s.sum()).item():.3f}, {(s[-1]/s.sum()).item():.3f}")
            
            sorted_s = torch.sort(s, descending=True).values  # 내림차순 정렬
            cumsum_s = torch.cumsum(sorted_s, dim=0)  # 누적 합
            
            total_sum = sorted_s.sum()
            k = int(0.025 * len(s))
            topk_sum = sorted_s[:k].sum()
            percentage = (topk_sum / total_sum) * 100
            
            print(f"상위 10개의 고유값이 차지하는 비율: {percentage:.2f}%")
            
            # threshold = 0.5 * sorted_s.sum()  # 전체 합의 90%

            # count = torch.searchsorted(cumsum_s, threshold, right=True).item() + 1  # 개수 찾기
            # print(f"90%를 차지하는 eigenvalue 개수: {count}/{len(s)}")



##### Hessians-Llama-2-7b-6144 #####


  2%|▎         | 1/40 [00:00<00:14,  2.61it/s]

## layer0, qkv ##
상위 10개의 고유값이 차지하는 비율: 97.85%
## layer0, o ##
상위 10개의 고유값이 차지하는 비율: 95.16%
## layer0, up ##
상위 10개의 고유값이 차지하는 비율: 50.61%
## layer0, down ##
상위 10개의 고유값이 차지하는 비율: 53.66%


  5%|▌         | 2/40 [00:00<00:16,  2.29it/s]

## layer1, qkv ##
상위 10개의 고유값이 차지하는 비율: 90.70%
## layer1, o ##
상위 10개의 고유값이 차지하는 비율: 62.09%
## layer1, up ##
상위 10개의 고유값이 차지하는 비율: 30.62%
## layer1, down ##
상위 10개의 고유값이 차지하는 비율: 99.87%


...

## layer29, qkv ##
상위 10개의 고유값이 차지하는 비율: 26.84%
## layer29, o ##
상위 10개의 고유값이 차지하는 비율: 21.09%
## layer29, up ##
상위 10개의 고유값이 차지하는 비율: 20.43%
## layer29, down ##
상위 10개의 고유값이 차지하는 비율: 40.56%


 78%|███████▊  | 31/40 [00:12<00:04,  1.93it/s]

## layer30, qkv ##
상위 10개의 고유값이 차지하는 비율: 24.46%
## layer30, o ##
상위 10개의 고유값이 차지하는 비율: 31.86%
## layer30, up ##
상위 10개의 고유값이 차지하는 비율: 22.73%
## layer30, down ##
상위 10개의 고유값이 차지하는 비율: 69.85%


100%|██████████| 40/40 [00:13<00:00,  3.03it/s]

## layer31, qkv ##
상위 10개의 고유값이 차지하는 비율: 26.98%
## layer31, o ##
상위 10개의 고유값이 차지하는 비율: 43.09%
## layer31, up ##
상위 10개의 고유값이 차지하는 비율: 26.78%
## layer31, down ##
상위 10개의 고유값이 차지하는 비율: 85.07%


In [11]:
import torch

# 값의 편차가 큰 대칭 행렬 생성
A = torch.tensor([[1000.0, 2.0, 3.0], 
                  [2.0, 0.01, 4.0], 
                  [3.0, 4.0, 0.0001]])

# LDL 분해 수행
L, D, _ = torch.linalg.ldl(A)

print("L 행렬:\n", L)
print("D 행렬:\n", D)


AttributeError: module 'torch.linalg' has no attribute 'ldl'

In [12]:
import numpy as np
import scipy.linalg

# 값의 편차가 큰 대칭 행렬 생성
A = np.array([[1000.0, 2.0, 3.0], 
              [2.0, 0.01, 4.0], 
              [3.0, 4.0, 0.0001]])

# LDL 분해 수행
L, D, perm = scipy.linalg.ldl(A, lower=True)

# 결과 출력
print("L 행렬:\n", L)
print("D 행렬:\n", D)


L 행렬:
 [[1.    0.    0.   ]
 [0.002 1.    0.   ]
 [0.003 0.    1.   ]]
D 행렬:
 [[ 1.000e+03  0.000e+00  0.000e+00]
 [ 0.000e+00  6.000e-03  3.994e+00]
 [ 0.000e+00  3.994e+00 -8.900e-03]]


In [15]:
import torch

def ldl_decomposition(A):
    # Cholesky 분해 수행 (A = L L^T)
    L_cholesky = torch.linalg.cholesky(A)

    # D 행렬: Cholesky 분해에서 얻은 L의 대각 원소의 제곱
    D = torch.diag(torch.diag(L_cholesky)**2)

    # L 행렬: Cholesky 행렬을 정규화하여 얻음
    L = L_cholesky / torch.diag(L_cholesky).reshape(-1, 1)

    return L, D

# 대칭 행렬 (값의 편차가 큰 경우)
A = torch.tensor([[1.0, 2.0, 3.0], 
                  [2.0, 1000, 4.0], 
                  [3.0, 4.0, 0.0001]])

# LDL 분해 수행
L, D = ldl_decomposition(A@A.T)

# 결과 출력
print("L 행렬:\n", L)
print("D 행렬:\n", D)


L 행렬:
 tensor([[1.0000, 0.0000, 0.0000],
        [0.6387, 1.0000, 0.0000],
        [1.0338, 1.0112, 1.0000]])
D 행렬:
 tensor([[1.4000e+01, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 7.1029e+05, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 8.0875e+00]])
